In [1]:
import requests 
import pandas as pd 
import json 
import numpy as np
import pprint
from bs4 import BeautifulSoup

In [2]:
#creating a header file to 
header = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}   

## Practice webscrape with a single game to use the same code in an extended loop

In [20]:
# created a seperate url to webscrape all data from a single game so i can loop it
# for a large set of game


urlr = 'https://www.metacritic.com/game/xbox-one/red-dead-redemption-2'
response_soupr = requests.get(urlr , headers = header)
soupr = BeautifulSoup(response_soupr.text, 'html.parser')

In [21]:
## I went to the website to find what I want
## then keep going up in class until I find something with a unique title
title = soupr.findAll('div', {'class' : "product_title"})

# now we have a much smaller section to look through and grab the piece of info we ashley care about
title[0]('h1')[0].text

# i continued this process for each individual topic i wanted to find on the specific page
# seperating all of them in individual blocks as they all have a unique method to the final result

'Red Dead Redemption 2'

In [22]:
metascores = soupr.findAll('div', {'class' : "metascore_wrap highlight_metascore"})

metascores[0].span.text

'97'

In [23]:
metascore_review = soupr.findAll('span', {'class' : "count"})
metascore_reviews = metascore_review[0].text.replace('\n', '').split()
metascore_reviews[3]

'33'

In [24]:
user_scores = soupr.findAll('div', {'class' : "userscore_wrap feature_userscore"})

user_score = user_scores[0].text.replace('\n', ' ').split()[2]
user_score

'7.8'

In [25]:
user_review = soupr.findAll('div', {'class' : "details side_details"})
user_review[0].p.span.text

'Generally favorable reviews'

In [26]:
content_rating = soupr.findAll('li', {'class' : "summary_detail product_rating"})
content_rating[0].text.replace('\n', ' ').split()[1]

'M'

In [27]:
publisher = soupr.findAll('li', {'class' : "summary_detail product_genre"})
[' '.join(publisher[0].text.replace('\n', ' ').split()[1:])]

['Action Adventure, Open-World']

In [28]:
genres = soupr.findAll('li', {'class' : "summary_detail developer"})
' '.join(genres[0].text.replace('\n', ' ').split()[1:])

'Rockstar Games'

In [29]:
release_date = soupr.findAll('li', {'class' : "summary_detail release_data"})
' '.join(release_date[0].text.replace('\n', ' ').split()[2:])


'Oct 26, 2018'

In [5]:
# url and response for the general page that has the hyperlink for all of the games 
# split up into 9 different pages

# urlb = 'https://www.metacritic.com/browse/games/release-date/available/xboxone/metascore?page=0'
# response_soup = requests.get(urlb, headers = header)
# soup = BeautifulSoup(response_soup.text, 'html.parser')

In [213]:
# create a new empty list to store all hyperlinks
all_hyperlink = []

# create a function to get all game links with parameters of the url and header
def get_game_links_from_site(url, header):
    response_soup = requests.get(url, headers = header)
    soup = BeautifulSoup(response_soup.text, 'html.parser')

    # game tablle is the block of information for all of the games
    game_table = soup.find_all('ol', class_="list_products list_product_condensed")
    
    # game is the box of data that has individual games in the game_table
    for game in game_table:
        
        # create a new variable to loop through to obtain the hyperlink in each block
        game_links_html = game.find_all('a', href=True)
        
        # this block is to test if the code is reading the url by giving it a sleep timer 
        # so it can stop for 5 seconds and continue again afterwards
        # if it goes through this block it will just call to the individual block again
        if game_links_html == []:
            time.sleep(5)
            game_links_html = game.find_all('a', href=True)
        
        # it continues onto this whether it goes into the if statement or not
        # will look through each block in game_links_html and appends
        # that hyperlink into my list of all hyperlinks
        for game_link in game_links_html:
            all_hyperlink.append(game_link['href'])
           # print(game_link['href'])
    return all_hyperlink





In [ ]:
# now the only thing left is to go through all the pages in the website since there were 
# 9 pages (page 0 to 8) and the only thing that changes is the page= 
# I just needed to change the page number and loop throguh it and it will append all game hyperlinks to our list 

for i in range(0, 9):
    url = 'https://www.metacritic.com/browse/games/release-date/available/xboxone/metascore?page={}'.format(i)        
    get_game_links_from_site(url, header)

In [3]:
# creating a list of dictionaries that will later convert into a dataframe

metacritic_dataframe_list = []

# create a function to append all the individual items for every game into a dictionary and 
# appending the dictionary into the list. Only parameter that will change is the url which
# is just coming from the list of urls from the previous function

def get_data_from_each_game (url):   
    
    response_soup = requests.get(url, headers = header)
    soup = BeautifulSoup(response_soup.text, 'html.parser')

    title = soup.findAll('div', {'class' : "product_title"})

    metascores = soup.findAll('div', {'class' : "metascore_wrap highlight_metascore"})

    metascore_review = soup.findAll('span', {'class' : "count"})

    user_scores = soup.findAll('div', {'class' : "userscore_wrap feature_userscore"})

    user_review = soup.findAll('div', {'class' : "details side_details"})

    content_rating = soup.findAll('li', {'class' : "summary_detail product_rating"})

    genres = soup.findAll('li', {'class' : "summary_detail product_genre"})

    developer = soup.findAll('li', {'class' : "summary_detail developer"})

    release_date = soup.findAll('li', {'class' : "summary_detail release_data"})

    # needed to create a try and exccept here because the  user review count is different 
    # if there are reviews as opposed to if there arent. so i used a try and except based 
    # on the error i got and i can output the appropriate dictionary for the gsme
    
    try:
        metacritic_dict = {
            'title': title[0].h1.text,
            'metacritic_score': metascores[0].span.text,
            'metacritic_review_count': metascore_review[0].text.replace('\n', '').split()[3],
            'user_score': user_scores[0].text.replace('\n', ' ').split()[2],
            'user_review_count': user_review[0].p.a.text.split()[0],
            'content_rating': content_rating[0].text.replace('\n', ' ').split()[1],
            'developer':' '.join(developer[0].text.replace('\n', ' ').split()[1:]),
            'genres': [' '.join(genres[0].text.replace('\n', ' ').split()[1:])],
            'release_date': ' '.join(release_date[0].text.replace('\n', ' ').split()[2:])
        }
    except AttributeError:
        metacritic_dict = {
            'title': title[0].h1.text,
            'metacritic_score': metascores[0].span.text,
            'metacritic_review_count': metascore_review[0].text.replace('\n', '').split()[3],
            'user_score': user_scores[0].text.replace('\n', ' ').split()[2],
            'user_review_count': user_review[0].p.span.text,
            'content_rating': content_rating[0].text.replace('\n', ' ').split()[1],
            'developer':' '.join(developer[0].text.replace('\n', ' ').split()[1:]),
            'genres': [' '.join(genres[0].text.replace('\n', ' ').split()[1:])],
            'release_date': ' '.join(release_date[0].text.replace('\n', ' ').split()[2:])
        }
    
    # appending the dictionary to the list
    metacritic_dataframe_list.append(metacritic_dict)
    
    return metacritic_dataframe_list
    

In [21]:
# problems loading the url in random cases, created a try and catch to catch any errors
# but continue past them. stored the errors in its own list that i can run through again
# until the list of errors is empty

errors = []

# goes through all the links in the hyperlinks list and edits the url accordingly then
# runs the function for that url to get the games

for link in all_hyperlink:
    try: 
        urlr = 'https://www.metacritic.com{}'.format(link)
        get_data_from_each_game(urlr)
    except IndexError:
        errors.append(link)

In [10]:
# had to create a second list of errors to catch problems when going through the 
# original list of errors

new_errors = []

for link in errors:
    try: 
        urlr = 'https://www.metacritic.com{}'.format(link)
        get_data_from_each_game(urlr)
    except IndexError as e:
        new_errors.append(link)

In [13]:
# this is to go through the new_errors list and will keep repeating with the previous code
# until there are no errors

errors = []

for link in new_errors:
    try: 
        urlr = 'https://www.metacritic.com{}'.format(link)
        get_data_from_each_game(urlr)
    except IndexError as e:
        errors.append(link)

In [1]:
# used pickle to store the lists i created since the function and loops took a long time
# to run i didnt want to have to run them over and over. 

import pickle

In [2]:
# these will open up the files i created

with open('hyperlink_list', 'rb') as handle:
    all_hyperlink = pickle.load(handle)
    
with open('error_urls', 'rb') as handle:
    errors = pickle.load(handle)

with open('working_list', 'rb') as handle:
    metacritic_dataframe_list = pickle.load(handle)

In [75]:
# these will store the files

with open('hyperlink_list', 'wb') as handle:
    pickle.dump(all_hyperlink, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('working_list', 'wb') as handle:
    pickle.dump(metacritic_dataframe_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('error_urls', 'wb') as handle:
    pickle.dump(errors, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# generated a dataframe with the list of dictionaries 
df = pd.DataFrame(metacritic_dataframe_list)

In [52]:
# all the numbers came back as strings so i had to change it. but in user score and 
# user review count they had 'tbd or 'No user score yet' respectively so i had to change those
# to nan so i could convert everything else into a float/int


df = pd.to_numeric(df, errors = 'ignore')

In [16]:
df.to_csv('metacritic_reviews.csv')

